Rodando Modelo Autorregressivo de Defasagem Distribuída

In [ ]:
# Carregando biblioteca
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ardl import ARDL, ardl_select_order
import numpy as np
from scipy import stats

In [ ]:
# Carregando base de dados do modelo
dados_estacionarios = pd.read_excel('/content/dados_estacionarios.xlsx',parse_dates=['data'],index_col='data')
print(df.head())

            acum_capital_diff  cap_instalada_diff  ...  part_lucros  div_capital
data                                               ...                          
2010-02-01           -3.78638           -0.145606  ...     1.179053        18.98
2010-03-01            6.31775            1.254408  ...     1.192629         9.79
2010-04-01            3.78269            0.554450  ...     1.220305        12.44
2010-05-01            1.73056           -0.045475  ...     1.197103        23.28
2010-06-01           -3.20600           -0.045366  ...     1.203125        16.09

[5 rows x 5 columns]


In [ ]:
# Garantir que a coluna "data" seja datetime e indice (opcional)
# dados_estacionarios['data'] = pd.to_datetime(dados_estacionarios['data'])
# dados_estacionarios = dados_estacionarios.set_index('data')

In [ ]:
# Definir variaveis do modelo
y = dados_estacionarios['acum_capital_diff'] #variavel dependente
x = dados_estacionarios[['cap_instalada_diff', 'juros_capital_diff', 'part_lucros', 'div_capital']]

In [ ]:
# Seleção dos Lags do modelo (MODELO RESTRITO SELECIONADO AUTOMATICAMENTE)
# -> maxlag define até quantas defasagens testar
# -> trend pode ser 'n' (sem constante), 'c'(constante), 'ct' (constante + tendência)
sel = ardl_select_order(
    endog=y,
    exog=x,
    maxlag=5,      # defasagens máximas da variável dependente
    maxorder=5,    # defasagens máximas das variáveis explicativas
    ic='aic',
    trend='c'
)

print(sel.model)           # estrutura do modelo (ordens ARDL escolhidas)
ardl_res = sel.model.fit() # estimar
print(ardl_res.summary())  # tabela completa de resultados

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


                              ARDL Model Results                              
Dep. Variable:      acum_capital_diff   No. Observations:                  185
Model:                  ARDL(3, 3, 0)   Log Likelihood                -535.553
Method:               Conditional MLE   S.D. of innovations              4.589
Date:                Sun, 07 Sep 2025   AIC                           1091.105
Time:                        02:00:17   BIC                           1123.146
Sample:                    05-01-2010   HQIC                          1104.094
                         - 06-01-2025                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.8817      0.796      1.108      0.270      -0.690       2.453
acum_capital_diff.L1     -0.4127      0.075     -5.534      0.000      -0.560      -0.265
acum_cap

In [ ]:
# Estimação manual do ARDL com variáveis adicionais em termos contemporâneos (MODELO COMPLETO)
modelo_ampliado = ARDL(
    endog=y,  # acum_capital_diff
    lags=3,   # 3 defasagens da dependente
    exog=df[['cap_instalada_diff', 'juros_capital_diff', 'part_lucros', 'div_capital']],
    order={'cap_instalada_diff': 3,  # 3 defasagens para capacidade
           'juros_capital_diff': 0,  # sem defasagens para juros
           'part_lucros': 0,         # sem defasagens para lucros
           'div_capital': 0},        # sem defasagens para dividendos
    trend='c'
)
resultados_ampliado = modelo_ampliado.fit()
print(resultados_ampliado.summary())

                               ARDL Model Results                              
Dep. Variable:       acum_capital_diff   No. Observations:                  185
Model:             ARDL(3, 3, 0, 0, 0)   Log Likelihood                -534.932
Method:                Conditional MLE   S.D. of innovations              4.573
Date:                 Sun, 07 Sep 2025   AIC                           1093.864
Time:                         02:39:39   BIC                           1132.312
Sample:                     05-01-2010   HQIC                          1109.450
                          - 06-01-2025                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.9338      0.821      1.137      0.257      -0.687       2.555
acum_capital_diff.L1     -0.4071      0.075     -5.434      0.000      -0.555      -0.259


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Rodando manualmente o teste de cointegração de Bounds. Isso deveria ser feito utilizando a biblioteca statsmodels no entanto o google colab não conseguiu acessar a função necessaria para fazer.

In [ ]:
#Teste manual de cointegração de Bounds

def bounds_test(f_statistic, n_observations, k_variables, case=3):
    """
    Implementação manual do teste de cointegração de Bounds
    baseado em Pesaran et al. (2001)

    case: 1 = sem constante ou tendência
          2 = com constante, sem tendência
          3 = com constante e tendência
    """
    # Valores críticos aproximados para o teste de Bounds (Pesaran et al., 2001)
    # Estes são valores aproximados - para pesquisa séria use valores exatos da tabela
    critical_values = {
        1: {  # Case 1: No intercept or trend
            'I0': {0.01: 4.94, 0.05: 3.62, 0.10: 3.02},
            'I1': {0.01: 5.58, 0.05: 4.18, 0.10: 3.62}
        },
        2: {  # Case 2: Restricted intercept, no trend
            'I0': {0.01: 5.33, 0.05: 3.88, 0.10: 3.22},
            'I1': {0.01: 6.18, 0.05: 4.61, 0.10: 3.99}
        },
        3: {  # Case 3: Unrestricted intercept, no trend (seu caso)
            'I0': {0.01: 6.36, 0.05: 4.65, 0.10: 3.89},
            'I1': {0.01: 7.27, 0.05: 5.47, 0.10: 4.61}
        }
    }

    # Determinar valores críticos baseados no caso
    cv_i0 = critical_values[case]['I0']
    cv_i1 = critical_values[case]['I1']

    # Realizar o teste
    results = {
        'F-statistic': f_statistic,
        'Critical Values - I(0)': cv_i0,
        'Critical Values - I(1)': cv_i1
    }

    # Interpretação
    if f_statistic > cv_i1[0.01]:
        results['Conclusion'] = 'Cointegração ao nível de 1% de significância'
    elif f_statistic > cv_i1[0.05]:
        results['Conclusion'] = 'Cointegração ao nível de 5% de significância'
    elif f_statistic > cv_i1[0.10]:
        results['Conclusion'] = 'Cointegração ao nível de 10% de significância'
    elif f_statistic > cv_i0[0.01]:
        results['Conclusion'] = 'Resultado inconclusivo'
    else:
        results['Conclusion'] = 'Não há evidência de cointegração'

    return results

# Calcular a estatística F manualmente para o teste de cointegração
# Primeiro, precisamos extrair os resíduos do modelo ARDL e estimar um modelo de correção de erro

# 1. Extrair resíduos do modelo ARDL
residuals = ardl_res.resid

# 2. Estimar um modelo de correção de erro nos resíduos
# Δu_t = γu_{t-1} + ε_t
from statsmodels.regression.linear_model import OLS

# Criar variáveis para o modelo de correção de erro
delta_residuals = residuals.diff().dropna()
lagged_residuals = residuals.shift(1).dropna()

# Garantir que os índices estejam alinhados
common_index = delta_residuals.index.intersection(lagged_residuals.index)
delta_residuals = delta_residuals.loc[common_index]
lagged_residuals = lagged_residuals.loc[common_index]

# Estimar o modelo
ecm_model = OLS(delta_residuals, lagged_residuals)
ecm_results = ecm_model.fit()

# 3. Calcular a estatística F para testar H0: γ = 0 (não cointegração)
f_statistic = (ecm_results.tvalues[0] ** 2)  # Para regressão simples, F = t^2

# 4. Aplicar o teste de Bounds
k_variables = x.shape[1]  # Número de variáveis explicativas
bounds_result = bounds_test(f_statistic, len(y), k_variables, case=3)

print("Resultado do Teste de Cointegração de Bounds:")
print("=" * 50)
print(f"Estatística F: {bounds_result['F-statistic']:.4f}")
print("\nValores Críticos I(0):")
for level, value in bounds_result['Critical Values - I(0)'].items():
    print(f"  {level*100}%: {value:.3f}")
print("\nValores Críticos I(1):")
for level, value in bounds_result['Critical Values - I(1)'].items():
    print(f"  {level*100}%: {value:.3f}")
print(f"\nConclusão: {bounds_result['Conclusion']}")

Resultado do Teste de Cointegração de Bounds:
Estatística F: 182.9680

Valores Críticos I(0):
  1.0%: 6.360
  5.0%: 4.650
  10.0%: 3.890

Valores Críticos I(1):
  1.0%: 7.270
  5.0%: 5.470
  10.0%: 4.610

Conclusão: Cointegração ao nível de 1% de significância


/tmp/ipython-input-1125253286.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f_statistic = (ecm_results.tvalues[0] ** 2)  # Para regressão simples, F = t^2
